In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import strftime
import numpy as np

import scrape_url_list as s
import acquire as a

In [3]:
s.get_new_links()

NameError: name 'requests' is not defined